## Iris data

Iris-blomsterdatasettet er et multivariat datasett introdusert av den britiske statistikeren og biologen Ronald Fisher i sin artikkel fra 1936. Det har blitt et populært dataset for å teste ut nye metoder. Datasettet inneholder 3 klasser, hvor hver klasse refererer til en type irisplante. Her utforsker vi datasettet med forskjellige visualiseringsmetoder. 

Som alltid laster vi først inn pakkene vi trenger. 

In [ ]:
import numpy as np
import pandas as pd 
from sklearn.datasets import load_iris
from sklearn.decomposition import PCA 

import plotly.express as px
import plotly.graph_objects as go

Så leser vi inn data og forbereder den til analysen. 

In [ ]:
# Read the file into a variable iris_data
iris = load_iris()
# print(iris)
iris_data = pd.DataFrame(iris['data'], columns=iris['feature_names'])
iris_labels = iris.target_names[iris['target']]

# Print the first 5 rows of the data
iris_data.head()
iris_data.tail()


**En egenskap om gangen**:

Først ser vi på en egenskap i datasettet om gangen. 
Lag et histogram av alle kolonner i datasettet. Tips: Bruk [`plotly.express.histogram`](https://plotly.com/python/histograms/). 

In [ ]:
# Histogramer
for col in iris_data.columns:
    fig = px.histogram(iris_data[col],
                        x = col,
                        histnorm='probability density',
                        nbins=150
                       )
    
    fig.show()

Nå vil vi legge til en ekstra dimensjon ved å bruke farge. Fargelegg histogramene etter spesies. 

In [ ]:
# Histogramer farget etter spesies

for col in iris_data.columns:
    fig = px.histogram(iris_data[col],
                        x = col,
                        histnorm='probability density',
                        nbins=150,
                        color = iris_labels
                       )
    
    fig.show()

Så skal vi se på tre egenskaper samtidig. Lag en figur med sepal width på x-aksen, sepal length på y-aksen, fargelagt etter type irisplante, der vi har et punkt for hver plante i datasettet. Tips: Bruk [`plotly.express.scatter`](https://plotly.com/python-api-reference/generated/plotly.express.scatter.html). 

In [ ]:
# Scatterplot
fig = px.scatter(iris_data,
                x="sepal width (cm)", 
                y='sepal length (cm)',
                color = iris_labels
                )

fig.show()

I stedet for en scatterplott vil vi noen ganger heller se på 2-dimensjonal tetthet. Lag en 2-dimensjonal tetthetsfigur (contour plot) med `sepal width (cm)` på x-aksen og `sepal length (cm)` på y-aksen. Tips: Bruk [`plotly.express.density_contour`](https://plotly.com/python-api-reference/generated/plotly.express.density_contour.html). 

In [ ]:
# Tetthet
fig = px.density_contour(iris_data,
                x="sepal width (cm)", 
                y='sepal length (cm)',
                color = iris_labels
                )

fig.show()

Det blir litt enklere å lese hvis vi ikke bare har konturer, men fargelegger etter tetthet. For å gjøre det, kan vi bruke [`plotly.graph_objects.Figure.update_traces`](https://plotly.com/python-api-reference/generated/plotly.graph_objects.Figure.html#plotly.graph_objects.Figure.update_traces)-metoden med argumentene `contours_coloring="fill"` og for eksempel `colorscale = 'Blues'`. De argumentene kan være vanskelig å finne i dokumentasjonen og det er ofte nødvendig å søke på nettet for å finne slike løsninger. 

In [ ]:
# Tetthet
fig = px.density_contour(iris_data,
                x="sepal width (cm)", 
                y='sepal length (cm)',
                color = iris_labels
                )

fig.update_traces(contours_coloring="fill", 
                  colorscale = 'Blues')
fig.show()

Til slutt ser vi på to metoder for å se på alle variablene samtidig. Regn ut de første to prinsipale komponenter PC1 og PC2. Så lag en scatterfigur med PC1 på x-aksen, PC2 på y-aksen, fargelagt etter type irisplante. Tips: Bruk [`sklearn.decomposition.PCA`](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html). Som vi har sett tidligere er sklearn sine metoder er alle bygget opp på en lignende måte. Først må vi lage en modell (`init`-metode), så må vi tilpasse modellen til treningsdata (`fit`-metode) og til slutt kan vi bruke den til å predikere (`predict`-metode) for nye data eller transformere (`transform`-metode) nye data. 

In [ ]:
pca = PCA(n_components=2)
pca.fit(iris_data)
# pca_df = pd.DataFrame(pca.transform(iris_data.drop("species")), 
pca_df = pd.DataFrame(pca.transform(iris_data), 
                      columns=['PC1', 'PC2'])

print(pca_df)

# Scatterplot
fig = px.scatter(data_frame=pca_df, 
                 x='PC1', 
                 y='PC2', 
                 color=iris_labels,
                 labels={'color':'Type'})
fig.show()

Hvis vi har så få koordinater som her kan det være et alternativ til PCA å bruke parallele koordinater. Bruk parallele koordinater for å visualisere iris data. Tips: Bruk [`plotly.express.parallel_coordinates`](https://plotly.com/python-api-reference/generated/plotly.express.parallel_coordinates.html). 

In [ ]:
# Parallele koordinater
fig = px.parallel_coordinates(
        iris_data,
        # color = iris_labels,
        dimensions=["sepal length (cm)","sepal width (cm)","petal length (cm)","petal width (cm)"])

fig.show()

Fargelegg figuren med parallele koordinater etter type irisplante. Merk at [`plotly.express.parallel_coordinates`](https://plotly.com/python-api-reference/generated/plotly.express.parallel_coordinates.html) ikke er tenkt for å brukes med kategoriske farger. For å få det til likevel, så kan vi bruke en kontinuerlig fargeskala som er stykkevis konstant. Vi må også bruke en numerisk variabel for å fargelegge etter i stedet for den kontinuerlige variablen vi har brukt til nå. 

In [ ]:
fig = px.parallel_coordinates(
        iris_data,
        dimensions=["sepal length (cm)","sepal width (cm)","petal length (cm)","petal width (cm)"],
        color= iris["target"],
        color_continuous_scale=[(0.00, "#8DD3C7"),   (0.33, "#8DD3C7"),
                                (0.33, "#FFFFB3"), (0.67, "#FFFFB3"),
                                (0.67, "#BEBADA"),  (1.00, "#BEBADA")]
        )
                    


fig.update_layout(coloraxis_colorbar=dict(
    title="Spesies", 
    tickvals=[0.33, 1, 1.67], 
    ticktext=iris.target_names
))

fig.show()

## Museum data

Museum datasettet (data/museum.csv) inneholder data om museumsbesøk mellom januar 2014 og juni 2021. Data er hentet fra [Los Angeles open data](https://data.lacity.org/Arts-Culture/Museum-Visitors/trxm-jn3c). Målet er å lage en figur av utviklingen av museumsbesøk de siste 7 årene. 

Begynn ved å lese inn data og se på de første radene. 

In [ ]:
import pandas as pd
import plotly.express as px
museum_data = pd.read_csv("data/museum.csv")
museum_data.head()

Ved bruk av kolonnen `Month`, lag en ny kolonne `Date` som er har `dtype` `pandas.datetime`. Fjern deretter kolonnen `Month`. 

In [ ]:
# lag kolonnen Date
museum_data['Date'] = pd.to_datetime(museum_data["Month"])

# print(museum_data.head())
# fjern kolonnen Month
museum_data.drop("Month",axis=1,inplace=True)

museum_data.head()

Forandre data sånn at du har tre kolonner med dato, museum navn og antal besøkende. Tips: Bruk [`pandas.melt`](https://pandas.pydata.org/docs/reference/api/pandas.melt.html). Sorter deretter etter dato og museum navn. 

In [ ]:
# omformater data
# museum_reshaped = pd.melt(museum_data,id_vars=["Date"], value_vars = ["IAMLA",])
museum_copy = museum_data
mueseum_cols = museum_copy.drop("Date",axis=1).columns
museum_reshaped = pd.melt(museum_data,id_vars=["Date"], value_vars = mueseum_cols)
# sorter 
museum_reshaped["value"] = museum_reshaped["value"].astype(float) #litt usikker, rart at 6000 kommer før 10000
museum_reshaped.sort_values("value",inplace=True)

Se på de første radene av det nye datasettet. 

In [ ]:
museum_reshaped.head()

Velg ut en informativ figurtype og lag en figur som viser utviklingen av museumsbesøk i datasettet. 

In [ ]:
# Visualisering

fig = px.histogram(museum_reshaped,
                   x="Date",
                   y="value",
                    color = "variable"
                    )
    
fig.show()

Hva legger du merke til i figuren du lagde? Nevn minst tre ting. 

1. Popularitet

Noen museummer får mye mer trafikk en andre, det er **Avila Adobe** som får mest besøk, dersom de tar opp mest plass i grafen (grønn)

2. Nye museummer

Museum of social justice har nok åpnet i 2018, siden det finnes ikke data for dette i histogrammet, og det er N/A verider for år før 2018 for Museum of social justice hvis man ser direkete i dataframen

3. Corona

Corona viruset hadde en stor effekt på alle museumer, og tallene gikk ned drastisk. I tilig 2021 var det kun "America Tropical Interpretive Center som hadde besøkende

...

## Titanic data

Titanic datasettet (data/titanic.csv) har vi sett på flere ganger i løpet av kurset. Her skal vi lage noen flere visualiseringer som relaterer prisen til andre variabler. 

Les inn data og se på de første radene av datasettet. 

In [ ]:
# lese inn data og se på de første radene. 
import pandas as pd
titanic = pd.read_csv("data/titanic.csv")
titanic.head()


Lag en meningsfull figur som viser kjønn (`Sex`) og pris (`Fare`) av titanic-passasjerene.

Er det noen forskjeller mellom prisene som kvinner og men betaler?

In [ ]:
# Visualisering av kjønn og pris

import plotly.express as px
import plotly.graph_objects as go

# titanic_fixed = titanic[titanic["Fare"] < 400] #Removed the large outliers value since it made the graph a

fig = px.box(titanic, y="Fare", x='Sex', width=800, height=1000)
fig.update_layout(font=dict(size=36),
template="simple_white",)
fig.show()




It seems like women pay more on averge than males. Both have some pretty large outliers. 

Variasjonen i prisen er større blant kvinner enn blant menn. Typisk betaler kvinner mer enn menn. 

Vi har sett i forelesninen at det er store forskjeller mellom kjønn og klassen (`Pclass`). Kan det forklare forskjellen mellom kjønn og prisen? Lag en figur for å se på de tre variablene samtidig. 

På første forsøk er det kanskje ikke like lett å se forskjellen i alle klassene. Hva kan du gjøre for å få alle klassene til å vises like godt?

Er det fortsatt forskjeller mellom kjønn og pris? I så fall, er det de samme forskjellene som vi så på i spørsmål før?

In [ ]:
# Visualisering av kjønn, pris og klasse

titanic_copy = titanic
titanic_copy['Sex'] = titanic_copy['Sex'].map({'male': 1, 'female': 0})

fig = px.parallel_coordinates(
titanic_copy,
color = titanic_copy["Fare"],
dimensions=['Sex', 'Fare',"Pclass"])

fig.update_layout(font=dict(size=16),
title="Sex vs Fare vs Class | 1 = male, 0 = female",
template="plotly_white",
width=1700, height=700)

fig.show()

From the graph above, one can see that most men really paid below 50, while a lot of women also paid below 50. However, a fair amount of women paid between 50-100, and even 100-150, much higher than the amount of men who paid these prices. It seems regardless that these women paid higher prices, the Pclass for this price range sits at 1, so they got to first class, having to pay a little more. It is also shown that most men paid below 50, but ended up in 3rd or second class.

It is notable that a woman paid a price of 500,and this could drag the mean up, but considering the amount of entries is very high, it should not be that much of an issue. 